Sentiment analysis via logistic regression

In [ ]:
import pandas as pd
import json, gzip
from urllib.request import urlopen
import multiprocessing as mp
from cytoolz import *
from ftfy import fix_text

## Download and clean up text

In [ ]:
url = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Patio_Lawn_and_Garden_5.json.gz'

In [ ]:
data = gzip.decompress(urlopen(url).read())
data = data.splitlines()

In [ ]:
df = pd.DataFrame(json.loads(line) for line in data)

In [ ]:
df.groupby('overall').size()

Get rid of four-star reviews (too wishy-washy!)

In [ ]:
df = df[df['overall']!=4].copy()

In [ ]:
df['sentiment'] = ['good' if s>4 else 'bad' for s in df['overall']]

In [ ]:
pd.value_counts(df['sentiment'])

In [ ]:
with mp.Pool() as p:
    df['reviewText'] = p.map(fix_text, df['reviewText'])

## Parse review texts

In [ ]:
import spacy
from spacy import displacy
from spacy.tokens import Token
nlp = spacy.load('en', disable=['ner'])

In [ ]:
df['doc'] = list(nlp.pipe(df['reviewText']))

## Baseline classifier

In [ ]:
from sklearn.feature_extraction.text import *
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.dummy import *

In [ ]:
train,test = train_test_split(df.index,
                             test_size=0.2,
                             stratify=df['sentiment'],
                             random_state=42)

In [ ]:
model0 = make_pipeline(CountVectorizer(analyzer=identity),DummyClassifier('most_frequent'))
model0.fit(df.loc[train]['reviewText'], df.loc[train]['sentiment'])
model0.score(df.loc[test]['reviewText'], df.loc[test]['sentiment']) * 100.

## Logistic regression

In [ ]:
def tokens(doc):
    return [tok.lower_ for tok in doc]

df['tokens'] = df['doc'].apply(tokens)

In [ ]:
params = {'logisticregression__C': [0.01, 0.1, 1.0],
         'countvectorizer__min_df':[1,2,5],
         'countvectorizer__max_df':[0.5,0.75]}
grid = GridSearchCV(lr, params, n_jobs=-1, cv=3)
grid.fit(df.loc[train]['tokens'], df.loc[train]['sentiment'])

In [ ]:
grid.best_params_

In [ ]:
lr.set_params(**grid.best_params_)
lr.fit(df.loc[train]['tokens'], df.loc[train]['sentiment'])
lr.score(df.loc[test]['tokens'], df.loc[test]['sentiment']) * 100.

Extract features (i.e., words) with the strongest sentiment associations

In [ ]:
def print_top_feats(M, k=0):
    V = M.named_steps['countvectorizer'].get_feature_names()
    coef = M.named_steps['logisticregression'].coef_[0]
    order = coef.argsort()
    for w1, w2 in zip(order[-k:][::-1],order[:k]):
        print(f'{V[w1]:20s} {coef[w1]:6.3f} | {V[w2]:20s} {coef[w2]:6.3f}')

In [ ]:
print_top_feats(lr, 20)

Check out a review that the model gets wrong

In [ ]:
df.loc[8181]['sentiment'], lr.predict([df.loc[8181]['tokens']])[0]

In [ ]:
df.loc[8181]['reviewText']

In [ ]:
print(sorted(set(df.loc[8181]['tokens'])))

## Handling negation

In [ ]:
s1 = nlp('I would never recommend this product to anyone.')
displacy.render(s1, style='dep', jupyter=True)

Tag tokens that are modified by negative words (e.g., *not*, *never*)

In [ ]:
Token.set_extension('neg', default=False, force=True)

In [ ]:
for doc in df['doc']:
    for tok in doc:
        if tok.dep_ == 'neg':
            tok.head._.neg = True

In [ ]:
def negtokens(doc):
    return ['NOT:'+tok.lower_ if tok._.neg else tok.lower_ for tok in doc]
df['negtokens'] = df['doc'].apply(negtokens)

In [ ]:
print(df.loc[8181]['negtokens'])

In [ ]:
lr.fit(df.loc[train]['negtokens'], df.loc[train]['sentiment'])
lr.score(df.loc[test]['negtokens'], df.loc[test]['sentiment']) * 100.

In [ ]:
lr.predict([df.loc[8181]['negtokens']])[0]

In [ ]:
print_top_feats(lr, 20)

## Adding other modifiers (adjectives, adverbs)

In [ ]:
displacy.render(df.loc[8181]['doc'], style='dep', jupyter=True)

In [ ]:
def negify(tok):
    return 'NOT:' + tok.lower_ if tok._.neg else tok.lower_

def mods(doc):
    return [ negify(w.head) + '_' + negify(w) for w in doc 
            if w.dep_ in ['amod', 'advmod'] ] + \
            [ negify(w) for w in doc]

df['mods'] = df['doc'].apply(mods)

In [ ]:
print(df.loc[8181]['mods'])

In [ ]:
lr.fit(df.loc[train]['mods'], df.loc[train]['sentiment'])
lr.score(df.loc[test]['mods'], df.loc[test]['sentiment']) * 100.

In [ ]:
print_top_feats(lr, 25)

## Add in **all** dependency pairs

In [ ]:
def everything(doc):
    return [ negify(w.head) + '_' + negify(w) for w in doc  
            if w.head != w ] + \
            [ negify(w) for w in doc]

df['everything'] = df['doc'].apply(everything)

In [ ]:
lr.fit(df.loc[train]['everything'], df.loc[train]['sentiment'])
lr.score(df.loc[test]['everything'], df.loc[test]['sentiment']) * 100.

In [ ]:
print_top_feats(lr, 25)